# 성능 향상

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False)

# 열 모두 표시
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
sub_dir = "/content/drive/MyDrive/Colab Notebooks/bus_rush_hour/submission/" 
os.chdir(sub_dir)

FileNotFoundError: ignored

## 앙상블

In [ ]:
# 5개의 submission 파일명을 각각 '모델 번호_사용한 모델=임시 스코어.csv' 형식으로 변경한 뒤 다음 과정을 진행합니다.
# 예를 들어, 'model1_lgbm.csv' 파일의 임시 스코어가 2.29라면 'model1_lgbm=2.29.csv' 형식으로 변경하는 것입니다.

sub_list = [] # 작업 경로 안에 있는 파일 중 확장자가 .csv인 파일을 담을 리스트 생성
fname_list = [] # 확장자를 제외한 파일명을 담을 리스트 생성

for filename in os.listdir(): # 작업 경로 안에 있는 모든 파일의 리스트 불러오기
    fname, ext = os.path.splitext(filename) # filename을 파일명과 확장자로 분리
    if ext == '.csv': # 확장자가 .csv인 파일인 경우
        sub_list.append(filename) # filename을 sub_list에 추가
        fname_list.append(fname) # 파일명을 fname_list에 추가

In [ ]:
# 상관관계가 낮은 submission 파일들끼리 앙상블을 하면 과적합의 위험이 줄어들음

# 상관계수 행렬을 저장할 데이터프레임 생성
corr_df = pd.DataFrame()

for file in sub_list:
    # 각 submission 파일의 18~20_ride 변수를 sub_df에 저장
    sub_df = pd.read_csv(file, engine = 'python').iloc[:,1:]
    # 변수명을 파일의 이름으로 지정
    sub_df.columns = [str(file)]
    # sub_df를 corr_df에 병합
    corr_df = pd.concat([corr_df, sub_df], axis = 1)

# 상관계수 행렬 출력    
corr_df.corr()

In [ ]:
corr_list = np.array(corr_df.corr().mean(axis = 0))  

model_list = [] # 모델명을 담을 리스트 생성
public_rmse_list = [] # 각 submission 파일의 임시 스코어를 담을 리스트 생성

for fname in fname_list:
    model = fname.split('=')[0] # '=' 기호를 기준으로 모델명을 분리
    model_list.append(model) # 모델명을 model_list에 추가

    score = fname.split('=')[-1] # '=' 기호를 기준으로 임시 스코어를 분리
    score = float(score) # 실수형으로 변환
    public_rmse_list.append(score) # 임시 스코어를 public_rmse_list에 추가
    
# model_list, public_rmse_list, corr_list를 변수로 하는 데이터프레임 생성
score_df = pd.DataFrame({'model': model_list, 'public_rmse': public_rmse_list,
                         'cor': corr_list})
score_df

In [ ]:
plt.figure(figsize=(10,5))
# 점의 사이즈가 30인 산점도 그래프
graph = sns.scatterplot(x="cor", y="public_rmse", data=score_df, s=30)
# 각 점에 대한 모델명 표시
for idx in range(0, score_df.shape[0]):
    graph.text(score_df.cor[idx]+0.00005 , score_df.public_rmse[idx]-0.00003, 
               score_df.model[idx], horizontalalignment='left', 
               size='medium', color='black', weight='semibold')

# x축 범위 지정
plt.xlim((score_df.cor.min()-0.001, score_df.cor.max()+0.001))
# y축 범위 지정
plt.ylim((score_df.public_rmse.min()-0.005, score_df.public_rmse.max()+0.005))
plt.grid() # 격자 무늬
plt.show()